### 🇯🇵 [JP]

---

#### 📌 概要

各試合（`matchid`）で**ピックされたチャンピオン一覧を保存**する作業です。

- `.rofl` リプレイファイルではなく  
- **CSVファイル**にある `matchid` リストを読み込み  
- 各試合に**参加したチャンピオン名**を抽出して保存します。

In [1]:
# [1] 初期設定
import os, json, csv
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

# 作業ディレクトリをプロジェクトルートに移動
PROJECT_ROOT = Path(__file__).parent.parent if "__file__" in dir() else Path.cwd().parent
os.chdir(PROJECT_ROOT)
print(f"作業ディレクトリ: {Path.cwd()}")

from autoLeague.replays.scraper import ReplayScraper
from autoLeague.dataset.riotapi import RiotAPI

load_dotenv("RiotAPI.env")
API_KEY = os.environ.get("API_KEY")

# ── 共通パス設定(Set common base path) ─────────────────────────────────────
USER_HOME = Path(os.environ["USERPROFILE"])    # C:\Users\<アカウント名>

# USER_HOME依存パス
REPLAY_DIR  = USER_HOME / "Documents/League of Legends/Replays"
SCRAPER_DIR = USER_HOME / "Desktop/LoL_WorkSp_win/pyLoL-_WorkSp/pyLoL-v2/autoLeague/replays"

# 固定パス
GAME_DIR       = Path(r"C:\Riot Games\League of Legends\Game")
SAVE_DIR       = Path(r"G:\マイドライブ\iput 授業\LoL_strorage\storage")
MATCH_INFO_DIR = Path(r"G:\マイドライブ\iput 授業\LoL_strorage\match_info")

ra = RiotAPI(api_key=API_KEY)
rs = ReplayScraper(game_dir=str(GAME_DIR), replay_dir=str(REPLAY_DIR), save_dir=str(SAVE_DIR), scraper_dir=str(SCRAPER_DIR), replay_speed=27, region="KR")

作業ディレクトリ: c:\Users\lapis\Desktop\LoL_WorkSp_win\pyLoL-_WorkSp\pyLoL-v2
LCU API initialized - port: 57328
Current number of replays:  190


In [2]:
# [2] matchid CSVファイルの統合
# ファイルパスリスト(List of file paths)
file_paths = [
    "matchids/sample_challenger_patch25_13_1.csv"
]

# 全ファイルを読み込んで1つにマージ(Read all files and merge into one DataFrame)
df_list = [pd.read_csv(path) for path in file_paths]
merged_df = pd.concat(df_list, ignore_index=True)

# 重複削除：matchid列基準 (Remove duplicates based on 'matchid' column)
if 'matchid' in merged_df.columns:
    merged_df = merged_df.drop_duplicates(subset='matchid')
else:
    merged_df = merged_df.drop_duplicates()

# 結果保存(Save the result)
os.makedirs("data", exist_ok=True)
merged_df.to_csv("data/match_champions(25.24).csv", index=False)

In [3]:
# [2.5] REPLAY_DIRに存在するmatchidのみ残す
# .roflファイル名からmatchidを抽出し、CSVとマッチングしてフィルタリング

# REPLAY_DIR内の.roflファイル一覧を取得
rofl_files = list(REPLAY_DIR.glob("*.rofl"))
print(f"REPLAY_DIR内の.roflファイル数: {len(rofl_files)}")

# .roflファイル名からmatchidを抽出（例: JP1-552649104.rofl → JP1_552649104）
# ファイル名のハイフンをアンダースコアに変換してmatchid形式に合わせる
rofl_matchids = set()
for rofl_path in rofl_files:
    filename = rofl_path.stem  # 拡張子なしのファイル名（例: JP1-552649104）
    matchid = filename.replace("-", "_")  # ハイフン→アンダースコア変換
    rofl_matchids.add(matchid)

print(f"抽出されたmatchid数: {len(rofl_matchids)}")
print(f"サンプル: {list(rofl_matchids)[:3]}")  # 確認用に3件表示

# CSVを読み込み、rofl_matchidsに存在するもののみフィルタリング
csv_path = "data/match_champions(25.24).csv"
df = pd.read_csv(csv_path)
original_count = len(df)

# matchid列でフィルタリング
df_filtered = df[df["matchid"].isin(rofl_matchids)]
filtered_count = len(df_filtered)

# 結果を保存
df_filtered.to_csv(csv_path, index=False)

print(f"フィルタリング完了:")
print(f"  元のmatchid数: {original_count}")
print(f"  REPLAY_DIRに存在するmatchid数: {filtered_count}")
print(f"  除外されたmatchid数: {original_count - filtered_count}")

REPLAY_DIR内の.roflファイル数: 190
抽出されたmatchid数: 190
サンプル: ['JP1_556025634', 'JP1_556331133', 'JP1_555629794']
フィルタリング完了:
  元のmatchid数: 190
  REPLAY_DIRに存在するmatchid数: 190
  除外されたmatchid数: 0


In [4]:
# [2.6] タイムラインデータの取得・保存
# matchidリストからRiot APIでタイムラインデータを取得し、JSONファイルとして保存

TIMELINE_DIR = Path("data/timeline")

# フィルタリング後のmatchidリストを取得
matchid_list = df_filtered["matchid"].tolist()
print(f"タイムラインを取得するmatchid数: {len(matchid_list)}")

# タイムラインデータを取得・保存
ra.save_timeline_data_from_matchlist(
    matchids=matchid_list,
    output_dir=str(TIMELINE_DIR)
)

タイムラインを取得するmatchid数: 190
Total match_ids: 190


Saving timeline data: 100%|██████████| 190/190 [03:11<00:00,  1.01s/it]

Completed! Saved: 190, Errors: 0


In [ ]:
# [2.7] マッチデータの取得・保存
# matchidリストからRiot APIでマッチデータを取得し、JSONファイルとして保存
# ※ DatasetBuilder（07_vision_score.ipynb）で勝敗情報取得に使用

MATCH_DIR = Path("data/match")

# フィルタリング後のmatchidリストを取得
matchid_list = df_filtered["matchid"].tolist()
print(f"マッチデータを取得するmatchid数: {len(matchid_list)}")

# マッチデータを取得・保存
ra.save_match_json_from_matchlist(
    matchids=matchid_list,
    output_dir=str(MATCH_DIR)
)

In [5]:
# [3] チャンピオン名取得・保存
ra.save_champnames_from_matches_without_rofl(input_csv_path='data/match_champions(25.24).csv',
                                             output_csv_path='data/match_champions_dict(25.24).csv')


Fetching match data: 100%|██████████| 190/190 [05:59<00:00,  1.89s/it]


### 🇯🇵 [JP]

---

#### 🧹 STEP 2: 検出不可または頻繁に肖像画が変わるチャンピオンを除外

- YOLO-11ベースの**検出モデルが正しく認識できないチャンピオン**  
  （→ 学習データの問題）  
- または**パッシブ/スキルで肖像画が頻繁に変わるチャンピオン**  
  （→ ゲーム内構造の問題）  
を**データセットから除去するステップ**です。

---

### 🇺🇸 [EN]

---

#### 🧹 STEP 2: Filtering out problematic champion classes

- Excludes champions that the **YOLO-11–based model cannot reliably detect**  
  (due to training data limitations),  
- and champions whose **in-game portraits frequently change**  
  (due to passives or skill transformations).  
This step filters them **out of the dataset**.

In [9]:
# [4] YOLOモデル未対応チャンピオンを動的に検出
# match_champions_dict から全チャンピオン抽出 → yolo_supported_champions.csv と比較

YOLO_SUPPORTED_CSV = "models/yolo_supported_champions.csv"
MATCH_CHAMP_DICT_CSV = "data/match_champions_dict(25.24).csv"

# 1. YOLOモデルが対応しているチャンピオン一覧を読み込む
yolo_df = pd.read_csv(YOLO_SUPPORTED_CSV)
yolo_supported = set(yolo_df["champion_name"].tolist())

# 2. 試合データから全チャンピオンを抽出
match_df = pd.read_csv(MATCH_CHAMP_DICT_CSV)
champ_cols = [c for c in match_df.columns if c.startswith("champion_")]
all_champions_in_matches = set()
for col in champ_cols:
    all_champions_in_matches.update(match_df[col].dropna().unique())

# 3. YOLOモデル未対応チャンピオンを特定
unsupported_by_yolo = all_champions_in_matches - yolo_supported

# 4. 変身問題のあるチャンピオン（対応はしているが検出に問題あり）
TRANSFORM_PROBLEM_CHAMPIONS = {"Neeko", "Viego", "Yuumi"}

# 5. 最終的な除外リストを生成
EXCLUDED_CHAMPIONS = unsupported_by_yolo | TRANSFORM_PROBLEM_CHAMPIONS

print("=" * 60)
print("【YOLOモデル未対応チャンピオン】")
print(f"  {unsupported_by_yolo if unsupported_by_yolo else '(なし)'}")
print()
print("【変身問題のあるチャンピオン】")
print(f"  {TRANSFORM_PROBLEM_CHAMPIONS}")
print()
print("【最終除外リスト】")
print(f"  {EXCLUDED_CHAMPIONS}")
print(f"  合計: {len(EXCLUDED_CHAMPIONS)} チャンピオン")
print("=" * 60)

【YOLOモデル未対応チャンピオン】
  {'Nilah', 'Kayle', 'Yunara', 'Zaahen'}

【変身問題のあるチャンピオン】
  {'Neeko', 'Viego', 'Yuumi'}

【最終除外リスト】
  {'Nilah', 'Kayle', 'Yunara', 'Zaahen', 'Neeko', 'Viego', 'Yuumi'}
  合計: 7 チャンピオン


In [10]:
# [5] 除外チャンピオンを含む試合をフィルタリング
# ※ EXCLUDED_CHAMPIONS は前のセルで動的に生成済み

CSV_IN  = "data/match_champions_dict(25.24).csv"   # 元のCSV 
CSV_OUT = "data/filtered_matchids(25.24).csv"      # 結果保存パス

# 除外リストを表示（確認用）
print(f"除外対象チャンピオン: {EXCLUDED_CHAMPIONS}")
print(f"除外対象数: {len(EXCLUDED_CHAMPIONS)}")

ra.filter_matches_by_excluded_champions(CSV_IN, CSV_OUT, EXCLUDED_CHAMPIONS)

除外対象チャンピオン: {'Nilah', 'Kayle', 'Yunara', 'Zaahen', 'Neeko', 'Viego', 'Yuumi'}
除外対象数: 7
Filtered 51 / 111 match_ids saved to data/filtered_matchids(25.24).csv


In [ ]:
# [6] プレイヤーランク情報取得 → 平均スコア計算 → キルイベント抽出 → 上位N試合抽出
MATCH_AVGSCR_CSV_PATH = "data/match_avg_score(25.24).csv"

### [6-1] match_champ_dict CSVファイルがある場合に使用 > match_infoフォルダに個別保存
# 注意: regionはRiotAPI初期化時に設定済みのため、パラメータ指定不要
ra.save_summoner_leagueinfo_from_csv(csv_in=CSV_IN, save_folder=MATCH_INFO_DIR, queue_type="RANKED_SOLO_5x5")

### match_champ_dict CSVファイルがない場合に使用
# ra.save_summoner_leagueinfo_of_replays(replay_dir = REPLAY_DIR, save_folder = MATCH_INFO_DIR)  # regionはRiotAPI初期化時に設定済み

### [6-2] 各試合のユーザー10人の平均スコアを計算してCSVに保存
ra.save_match_avg_score_from_json(match_info_dir=MATCH_INFO_DIR, output_csv_path=MATCH_AVGSCR_CSV_PATH)
matchid_list = pd.read_csv(MATCH_AVGSCR_CSV_PATH, usecols=["match_id"]).values.flatten().tolist()

### [6-3] 各試合の各時間帯（分単位）でキルイベントがthreshold以上の時間帯のみ抽出してJSONファイルに保存
ra.save_kill_events_from_matchlist(matchids=matchid_list, output_json_path="data/kill_events_timeline_from_all_matches.json", kill_events_threshold=4, max_workers=4)

### [6-4] 分（minute）10以上のみ残す
INPUT_JSON  = "data/kill_events_timeline_from_all_matches.json"
OUTPUT_JSON = "data/kill_events_timeline_filtered_10up.json"

with open(INPUT_JSON, "r", encoding="utf-8") as f:
    data = json.load(f)

filtered = {
    mid: [m for m in minutes if m >= 10]
    for mid, minutes in data.get("data", {}).items()
    if any(m >= 10 for m in minutes)
}

os.makedirs(os.path.dirname(OUTPUT_JSON), exist_ok=True)
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump({"data": filtered}, f, ensure_ascii=False, indent=2)

print(f"保存完了 → {OUTPUT_JSON}  (試合数: {len(filtered)})")

### [6-5] 上位N試合のみ残るようにフィルタリング
TOP_KILL_EVENTS_JSON = "data/top_kill_events_timeline_filtered_10up.json"
ra.save_top_matches_by_score(
    avg_score_csv_path=MATCH_AVGSCR_CSV_PATH,
    data_json_path=OUTPUT_JSON,
    output_path=TOP_KILL_EVENTS_JSON,
    top_n=1000
)

Total match_ids: 33, Already processed: 0, To process: 33
Starting to process 33 match_ids...
